In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [5]:
#Assign url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

#Retreive table from Wikipedia with read_html() method; this will be a list
wtable = pd.read_html(url, index_col=0,skiprows=1, attrs={"class":"wikitable"})  

#Table is contained in the first list element, then assigned to a variable for formatting
raw_table = wtable[0]

#Remove unassigned boroughs and rename columns 
test_table = raw_table[~raw_table[1].str.contains("Not assigned")].reset_index()
test_table.columns = ['PostalCode','Borough','Neighborhood']

#Iterate through rows and compare borough and neighbourhood values
for i in range(0,(test_table.shape[0])):
    if (test_table.iloc[i,:][1] != 'Not assigned') & (test_table.iloc[i,:][2] == 'Not assigned'):
        test_table.iloc[i, test_table.columns.get_loc('Neighborhood')] = test_table.iloc[i, test_table.columns.get_loc('Borough')]

#Group dataframe by postal code and borough to combine neighborhood values for one borough
df_toronto = test_table.groupby(['PostalCode', 'Borough'], as_index=False, sort=False).agg(', '.join)

df_toronto.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
df_toronto.shape

(103, 3)